# 第9章：从零开始构建复杂提示

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格，加载您的API密钥并建立 `get_completion` 辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程

恭喜您来到最后一章！现在是时候将所有知识融会贯通，学习如何**创建独特而复杂的提示**了。

在下文中，您将使用我们**推荐的复杂提示引导结构**。在本章的后续部分，我们将向您展示一些行业特定的提示，并解释这些提示是如何以类似方式构建的。

**注意：并非所有提示都需要包含以下复杂结构中的每个元素**。我们鼓励您尝试添加或删除某些元素，看看它们如何影响Claude的响应。通常，**最好是先使用许多提示元素让您的提示正常工作，然后再对其进行优化和精简**。

### 示例 - 职业教练聊天机器人

以下结构结合了多种提示工程元素，是构建复杂提示的一个良好起点。**某些元素的顺序很重要**，而另一些则不重要。我们会在最佳实践表明顺序重要时进行说明，但总的来说，**如果您遵循这个顺序，这将是构建一个出色提示的良好开端**。

对于以下示例，我们将为一个受控的角色扮演构建一个提示，其中Claude将扮演一个具有特定任务的情境角色。我们的目标是提示Claude扮演一个友好的职业教练。

阅读并运行下面的单元格，将各种提示元素编译成一个完整的提示。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 对话历史（这也可以作为API调用中前置的 `user` 和 `assistant` 消息添加）
HISTORY = """客户：给我两个社会学专业可能的职业。

乔：以下是社会学专业毕业生的两个潜在职业：

- 社会工作者 - 社会学为理解人类行为和社会系统提供了坚实的基础。通过额外的培训或认证，社会学学位可以使毕业生胜任社会工作者、个案经理、咨询师和社区组织者等角色，帮助个人和团体。

- 人力资源专员 - 社会学中对群体动力学和组织行为的理解适用于人力资源领域的职业。毕业生可以在招聘、员工关系、培训与发展、多元化与包容性以及其他人事职能中找到工作。对社会结构和制度的关注也支持公共政策、非营利组织管理和教育等相关职业。"""

# 第二个输入变量 - 用户的问题
QUESTION = "这两个职业中，哪个需要学士学位以上学历？"



######################################## 提示元素 ########################################

##### 提示元素 1：`user` 角色
# 确保您的 Messages API 调用中的消息数组始终以 `user` 角色开头。
# 上面定义的 `get_completion()` 函数会自动为您完成此操作。

##### 提示元素 2：任务上下文
# 告知 Claude 它应该扮演的角色，或者您希望它通过此提示承担的目标和总体任务。
# 最好将上下文放在提示正文的早期部分。
TASK_CONTEXT = "您将扮演一个名为乔的AI职业教练，由AdAstra Careers公司创建。您的目标是为用户提供职业建议。您将回复AdAstra网站上的用户，如果您不以乔的角色回应，他们会感到困惑。"

##### 提示元素 3：语气上下文
# 如果对交互很重要，请告诉 Claude 它应该使用什么语气。
# 此元素可能不是所有任务都必需的。
TONE_CONTEXT = "您应该保持友好的客户服务语气。"

##### 提示元素 4：详细任务描述和规则
# 详细说明您希望 Claude 执行的具体任务，以及 Claude 可能需要遵循的任何规则。
# 您也可以在此处为 Claude 提供一个“退出”选项，如果它没有答案或不知道如何回应。
# 最好将此描述和规则展示给朋友，以确保其逻辑清晰，并且任何模糊的词语都已明确定义。
TASK_DESCRIPTION = """以下是交互的一些重要规则：
- 始终保持乔（AdAstra Careers的AI）的角色
- 如果不确定如何回应，请说“抱歉，我没听懂。您能换个方式提问吗？”
- 如果有人问了不相关的问题，请说“抱歉，我是乔，我提供职业建议。今天有什么职业问题我可以帮助您吗？”"""

##### 提示元素 5：示例
# 为 Claude 提供至少一个它能模仿的理想响应示例。将其封装在 <示例></示例> XML 标签中。您可以随意提供多个示例。
# 如果您提供多个示例，请告知 Claude 这是什么示例，并将每个示例都包含在自己的 XML 标签集中。
# 示例可能是知识工作中让 Claude 表现出预期行为的最有效工具。
# 确保为 Claude 提供常见边缘情况的示例。如果您的提示使用了草稿本（scratchpad），提供草稿本应如何呈现的示例会很有效。
# 通常，示例越多越好。
EXAMPLES = """以下是标准交互中如何回应的示例：
<示例>
客户：你好，你是如何被创建的，你做什么？
乔：您好！我叫乔，由AdAstra Careers创建，旨在提供职业建议。今天有什么可以帮助您的吗？
</示例>"""

##### 提示元素 6：要处理的输入数据
# 如果提示中包含 Claude 需要处理的数据，请将其包含在此处相关的 XML 标签内。
# 您可以包含多条数据，但请务必将每条数据都包含在自己的 XML 标签集中。
# 此元素可能不是所有任务都必需的。顺序也灵活。
INPUT_DATA = f"""以下是问题之前的对话历史（用户和您之间）。如果没有历史，它可能为空：
<历史>
{HISTORY}
</历史>

以下是用户的问题：
<问题>
{QUESTION}
</问题>"""

##### 提示元素 7：即时任务描述或请求 #####
# “提醒”Claude 或明确告诉 Claude 它需要立即做什么来完成提示的任务。
# 这也是您放置其他变量（如用户问题）的地方。
# 重申 Claude 的即时任务通常没有坏处。最好在长提示的末尾这样做。
# 这将比将其放在开头产生更好的结果。
# 通常，将用户查询放在提示的底部附近也是一个好习惯。
IMMEDIATE_TASK = "您如何回应用户的问题？"

##### 提示元素 8：预认知（逐步思考）
# 对于多步骤任务，最好告诉 Claude 在给出答案之前逐步思考。
# 有时，您甚至可能需要说“在您给出答案之前...”以确保 Claude 首先执行此操作。
# 并非所有提示都必需，但如果包含，最好在长提示的末尾，紧接在最终的即时任务请求或描述之后。
PRECOGNITION = "在回应之前，请先思考您的答案。"

##### 提示元素 9：输出格式
# 如果您希望 Claude 的回应有特定的格式，请清楚地告诉 Claude 该格式是什么。
# 此元素可能不是所有任务都必需的。
# 如果您包含它，将其放在提示的末尾比放在开头更好。
OUTPUT_FORMATTING = "将您的回应放入 <响应></响应> 标签中。"

##### 提示元素 10：预填充 Claude 的回应（如果有）
# 一个用于以预填充的词语开始 Claude 的答案的空间，以引导 Claude 的行为或回应。
# 如果您想预填充 Claude 的回应，您必须将其放入 API 调用中的 `assistant` 角色。
# 此元素可能不是所有任务都必需的。
PREFILL = "[乔] <响应>"



######################################## 组合元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n助手回合")
print(PREFILL)

现在，让我们运行提示！运行下面的单元格，查看 Claude 的输出。

In [ ]:
print("\n------------------------------------- Claude 的回应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

### 示例 - 法律服务

**法律行业的提示可能相当复杂**，因为需要：
- 解析冗长的文档
- 处理复杂的主题
- 以非常特定的方式格式化输出
- 遵循多步骤的分析过程

让我们看看如何使用复杂的提示模板来构建一个针对特定法律用例的提示。下面，我们详细说明了一个法律用例的示例提示，其中我们要求Claude使用法律文件中的信息来回答有关法律问题。

我们**调整了一些元素的顺序**，以展示提示结构可以灵活多变！

**提示工程关乎科学的试错**。我们鼓励您混合搭配，调整元素位置（对于顺序不重要的元素），看看什么最适合您和您的需求。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 法律文件
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
动物健康产业在过去一年中卷入了一系列专利和商标诉讼。1994年，Barclay Slocum获得了胫骨平台水平截骨术（用于治疗犬颅十字韧带断裂）及其手术器械的专利。2006年，Slocum Enterprises 对 New Generation Devices 提起专利侵权诉讼，声称 New Generation 制造的 Unity Cruciate Plate 侵犯了 Slocum TPLO plate 的专利。然而，法院并未就专利侵权问题作出裁决，而是裁定其不具备管辖权，理由是在案件提起的州内销售的骨板数量很少，以及 Slocum Enterprises 维护的网站上提供的信息。2006年期间发生的其他专利纠纷涉及猫的激光去爪术、宠物识别芯片、猪疫苗和宠物“脱毛”工具的使用。
</search_result>
<search_result id=2>
在加拿大，不列颠哥伦比亚兽医协会对一名非兽医提起诉讼，声称他从事从马牙中切割或以其他方式移除钩子，以及使用电动和手动工具为马牙进行浮锉，并有偿提供建议和诊断，并自称有资格并愿意提供与这些活动相关的治疗。法院认为，立法机关通过《兽医专业法案》的意图是保护公众和动物，并进一步认为垄断性法规旨在保护公众。此外，法院得出结论，牙科的核心是与牙齿和牙龈健康相关；它不同于动物的美容和其他类型的护理；因此，它属于兽医实践的定义范围。该非兽医被禁止在没有兽医监督程序的情况下提供服务。
</search_result>
<search_result id=3>
2005年袭击美国墨西哥湾沿岸的卡特里娜飓风的后果，促使了自然灾害期间动物处理方式的改变。2006年，夏威夷、路易斯安那州和新罕布什尔州都颁布了法律，解决灾害期间动物护理问题，例如为宠物提供庇护所，并允许服务性动物与它们服务的人员一起安置。此外，国会于2006年通过并由总统签署了《宠物疏散和运输标准法案》，该法案要求州和地方应急准备机构在其疏散计划中包含如何在灾害发生时安置家庭宠物和服务性动物的信息。加利福尼亚州通过了一项法律，要求其紧急服务办公室、农业部以及其他参与灾害响应准备的机构制定一项计划，以应对灾害或重大紧急情况中服务性动物、牲畜、马科动物和家庭宠物的需求。
</search_result>
</search_results>"""

# 第二个输入变量 - 用户的问题
QUESTION = "关于飓风期间如何处理宠物，有什么法律规定吗？"



######################################## 提示元素 ########################################

##### 提示元素 1：`user` 角色
# 确保您的 Messages API 调用中的消息数组始终以 `user` 角色开头。
# 上面定义的 `get_completion()` 函数会自动为您完成此操作。

##### 提示元素 2：任务上下文
# 告知 Claude 它应该扮演的角色，或者您希望它通过此提示承担的目标和总体任务。
# 最好将上下文放在提示正文的早期部分。
TASK_CONTEXT = "您是一位专业的律师。"

##### 提示元素 3：语气上下文
# 如果对交互很重要，请告诉 Claude 它应该使用什么语气。
# 此元素可能不是所有任务都必需的。
TONE_CONTEXT = ""

##### 提示元素 4：要处理的输入数据
# 如果提示中包含 Claude 需要处理的数据，请将其包含在此处相关的 XML 标签内。
# 您可以包含多条数据，但请务必将每条数据都包含在自己的 XML 标签集中。
# 此元素可能不是所有任务都必需的。顺序也灵活。
INPUT_DATA = f"""这里是一些已编译的研究资料。请用它来回答用户的法律问题。
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### 提示元素 5：示例
# 为 Claude 提供至少一个它能模仿的理想响应示例。将其封装在 <example></example> XML 标签中。您可以随意提供多个示例。
# 如果您提供多个示例，请告知 Claude 这是什么示例，并将每个示例都包含在自己的 XML 标签集中。
# 示例可能是知识工作中让 Claude 表现出预期行为的最有效工具。
# 确保为 Claude 提供常见边缘情况的示例。如果您的提示使用了草稿本（scratchpad），提供草稿本应如何呈现的示例会很有效。
# 通常，示例越多越好。
EXAMPLES = """在您的回答中引用法律研究时，请使用包含搜索索引ID的方括号，后跟一个句号。将其放在引用句的末尾。正确的引用格式示例：

<examples>
<example>
此类犯罪的诉讼时效在10年后到期。[3]。
</example>
<example>
然而，当双方明确放弃保护时，该保护不适用。[5]。
</example>
</examples>"""

##### 提示元素 6：详细任务描述和规则
# 详细说明您希望 Claude 执行的具体任务，以及 Claude 可能需要遵循的任何规则。
# 您也可以在此处为 Claude 提供一个“退出”选项，如果它没有答案或不知道如何回应。
# 最好将此描述和规则展示给朋友，以确保其逻辑清晰，并且任何模糊的词语都已明确定义。
TASK_DESCRIPTION = f"""请对这个问题给出清晰、简洁的回答：

<问题>
{QUESTION}
</问题>

回答不应超过两段。如果可能，它应该以一个直接回答用户问题的单句结尾。但是，如果编译的研究资料中没有足够的信息来给出这样的答案，您可以拒绝回答并写“抱歉，我手头没有足够的信息来回答这个问题。”"""

##### 提示元素 7：即时任务描述或请求 #####
# “提醒”Claude 或明确告诉 Claude 它需要立即做什么来完成提示的任务。
# 这也是您放置其他变量（如用户问题）的地方。
# 重申 Claude 的即时任务通常没有坏处。最好在长提示的末尾这样做。
# 这将比将其放在开头产生更好的结果。
# 通常，将用户查询放在提示的底部附近也是一个好习惯。
IMMEDIATE_TASK = ""

##### 提示元素 8：预认知（逐步思考）
# 对于多步骤任务，最好告诉 Claude 在给出答案之前逐步思考。
# 有时，您甚至可能需要说“在您给出答案之前...”以确保 Claude 首先执行此操作。
# 并非所有提示都必需，但如果包含，最好在长提示的末尾，紧接在最终的即时任务请求或描述之后。
PRECOGNITION = "在您回答之前，请从研究资料中提取最相关的引用，并放入 <relevant_quotes> 标签中。"

##### 提示元素 9：输出格式
# 如果您希望 Claude 的回应有特定的格式，请清楚地告诉 Claude 该格式是什么。
# 此元素可能不是所有任务都必需的。
# 如果您包含它，将其放在提示的末尾比放在开头更好。
OUTPUT_FORMATTING = "将您的两段式回应放入 <answer> 标签中。"

##### 提示元素 10：预填充 Claude 的回应（如果有）
# 一个用于以预填充的词语开始 Claude 的答案的空间，以引导 Claude 的行为或回应。
# 如果您想预填充 Claude 的回应，您必须将其放入 API 调用中的 `assistant` 角色。
# 此元素可能不是所有任务都必需的。
PREFILL = "<relevant_quotes>"



######################################## 组合元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n助手回合")
print(PREFILL)

现在，让我们运行提示！运行下面的单元格，查看 Claude 的输出。

In [ ]:
print("\n------------------------------------- Claude 的回应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

如果您想在不更改上方任何内容的情况下，尝试课程中的提示，请一直滚动到课程笔记本的底部，访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 9.1 - 金融服务聊天机器人](#exercise-91---financial-services-chatbot)
- [练习 9.2 - 代码机器人](#exercise-92---codebot)

### 练习 9.1 - 金融服务聊天机器人
金融行业的提示也可能相当复杂，原因与法律提示类似。这是一个金融用例的练习，其中 Claude 用于**分析税务信息并回答问题**。就像法律服务示例一样，我们调整了一些元素的顺序，因为我们的解决方案提示采用不同的流程会更合理（然而，其他结构也同样适用）。

我们建议您仔细阅读变量内容（在本例中是 `{QUESTION}` 和 `{TAX_CODE}`），以了解 Claude 预期处理的内容。请务必在您的提示中直接引用 `{QUESTION}` 和 `{TAX_CODE}`（使用与其他示例类似的 f-string 语法），以便实际的变量内容能够被替换进去。

请根据您在前面复杂提示示例中看到的描述和示例，填写提示元素字段的内容。填写完所有您想填写的提示元素后，运行该单元格，查看拼接后的提示以及 Claude 的回应。

请记住，提示工程很少是纯粹的公式化，特别是对于大型和复杂的提示！重要的是要开发测试用例，并**尝试各种提示和提示结构，看看哪种最适合每种情况**。请注意，如果您*确实*更改了提示元素的顺序，您也应该记住更改 `COMBINE ELEMENTS` 部分中拼接的顺序。

In [ ]:
######################################## 输入变量 ########################################

# 第一个输入变量 - 用户的问题
QUESTION = "我有多长时间可以进行 83b 选举？"

# 第二个输入变量 - Claude 将用于回答用户问题的税法文件
TAX_CODE = """
(a)一般规则
如果，在提供服务时，财产被转让给除提供该服务的人以外的任何其他人，则以下两者中较早发生者：
(1)该财产的公允市场价值（不考虑任何非永久性限制），在拥有该财产受益权益的人的权利可转让或不再受实质性没收风险约束时的首次确定，超出
(2)为该财产支付的金额（如果有），
应计入提供该服务的人在拥有该财产受益权益的人的权利可转让或不再受实质性没收风险约束（以适用者为准）的第一个纳税年度的总收入中。如果该人在其对该财产的权利变得可转让或不再受实质性没收风险约束之前，通过公平交易出售或以其他方式处置该财产，则前一句不适用。
(b)选择在转让年度计入总收入
(1)一般规定
任何在提供服务时获得财产转让的人，可以选择在其获得该财产转让的纳税年度，将以下两者中较高者计入其总收入：
(A)该财产在转让时的公允市场价值（不考虑任何非永久性限制），超出
(B)为该财产支付的金额（如果有）。
如果作出此项选择，则本款(a)不适用于该财产的转让，并且如果该财产随后被没收，则不得就该没收允许任何扣除。
(2)选择
根据第(1)款对任何财产转让作出的选择，应以财政部长规定的方式作出，并且应在转让之日起不迟于30天内作出。除非经财政部长同意，否则该选择不得撤销。

(c)特殊规则
就本节而言——
(1)实质性没收风险
如果一个人对财产的完全享用权利取决于任何个人未来提供实质性服务，则该人在财产中的权利受实质性没收风险约束。

(2)财产的可转让性
一个人在财产中的权利仅当任何受让人在该财产中的权利不受实质性没收风险约束时才可转让。

(3)可能根据1934年证券交易法第16(b)节引起诉讼的销售
只要财产的盈利销售可能使某人根据1934年证券交易法第16(b)节受到诉讼，则该人在该财产中的权利是——
(A)受实质性没收风险约束的，并且
(B)不可转让的。
(4)为了确定个人在因提供服务而转让的财产中的基础，应适用与第72(w)节规则类似的规则。
(d)某些永不失效的限制
(1)估值
对于受其条款永不失效的限制约束的财产，并且该限制允许受让人仅以根据公式确定的价格出售该财产，则除非财政部长另有规定，否则按此确定的价格应被视为该财产的公允市场价值，并且举证责任应由财政部长承担。

(2)取消
如果，对于受其条款永不失效的限制约束的财产，该限制被取消，则除非纳税人证明——
(A)该取消并非补偿性的，并且
(B)如果该取消被视为补偿性的，则任何可能获得扣除的人将按照财政部长通过法规规定的方式，将该交易视为非补偿性的，
否则，该财产在取消时的公允市场价值（不考虑限制计算）超出以下两项之和的部分——
(C)该财产在取消前立即的公允市场价值（考虑限制计算），以及
(D)为取消支付的金额（如果有），
应被视为该取消发生纳税年度的补偿。
(e)本节的适用性
本节不适用于——
(1)第421节适用的交易，
(2)转入或转出第401(a)节所述信托的转让，或根据符合第404(a)(2)节要求的年金计划进行的转让，
(3)没有可轻易确定公允市场价值的期权的转让，
(4)根据在授予日期具有可轻易确定公允市场价值的期权行使而进行的财产转让，或
(5)第79节适用的团体定期人寿保险。
(f)持有期
在确定纳税人持有本款(a)适用的财产的期间时，仅应包括其对该财产的权利可转让或不再受实质性没收风险约束（以较早发生者为准）的首次开始的期间。

(g)某些交换
如果本款(a)适用的财产被交换为受与该交换中给予的财产实质性相似的限制和条件约束的财产，并且如果第354、355、356或1036节（或第1031节中与第1036节相关的部分）适用于该交换，或者如果该交换是根据转换特权行使的——
(1)该交换应为本款(a)的目的而被忽略，并且
(2)所收到的财产应被视为本款(a)适用的财产。
(h)雇主扣除
对于本节适用的财产转让或本款(d)所述限制的取消，根据第162节，为之提供服务的个人，应允许扣除一笔金额，该金额等于根据本款(a)、(b)或(d)(2)计入提供该服务的人的总收入的金额。该扣除应允许在该金额计入提供该服务的人的总收入的纳税年度或与该纳税年度结束的该人的纳税年度。

(i)合格股权赠与
(1)一般规定
就本分节而言——
(A)计入时间
如果合格股票被转让给根据本款对该股票作出选择的合格员工，则本款(a)应通过将根据本款确定的与该股票相关的金额计入员工在(B)款确定的纳税年度的收入中，而非本款(a)所述的纳税年度来适用。

(B)确定的纳税年度
本款确定的纳税年度是员工的纳税年度，包括以下最早的日期——
(i)该合格股票首次变得可转让的日期（仅就本条款而言，包括变得可转让给雇主），
(ii)员工首次成为被排除员工的日期，
(iii)发行该合格股票的公司任何股票首次在已建立的证券市场（由财政部长确定，但不包括任何市场，除非该市场被财政部长认定为本分节以外的本标题条款的已建立证券市场）上可轻易交易的日期，
(iv)员工在该股票中的权利可转让或不再受实质性没收风险约束（以较早发生者为准）的首次日期后的5年，或
(v)员工根据本款撤销（在财政部长规定的时间和方式下）对该股票的选择的日期。
(2)合格股票
(A)一般规定
就本款而言，“合格股票”一词，就任何合格员工而言，指该员工雇主公司中的任何股票，如果——
(i)该股票是——
(I)与期权行使相关，或
(II)作为限制性股票单位的结算，并且
(ii)该期权或限制性股票单位由公司授予——
(I)与作为员工提供服务相关，并且
(II)在该公司为合格公司的日历年度内。
(B)限制
如果员工在该股票中的权利首次变得可转让或不再受实质性没收风险约束时，可以将该股票出售给公司，或以其他方式从公司获得现金代替股票，则“合格股票”一词不包括任何此类股票。

(C)合格公司
就(A)(ii)(II)款而言——
(i)一般规定
“合格公司”一词，就任何日历年度而言，指任何公司，如果——
(I)该公司的任何股票（或该公司的任何前身公司的股票）在任何前一个日历年度内未在已建立的证券市场（根据第(1)(B)(iii)款确定）上可轻易交易，并且
(II)该公司有一项书面计划，根据该计划，在该日历年度内，至少80%的在美国（或美国任何属地）为该公司提供服务的所有员工被授予股票期权，或被授予限制性股票单位，并具有相同的权利和特权以获得合格股票。
(ii)相同的权利和特权
就第(i)(II)款而言——
(I)除第(II)和(III)分款另有规定外，股票权利和特权的确定应以与第423(b)(5)节类似的方式进行，
(II)员工不应仅仅因为可供所有员工获得的股份数量不相等而被视为不具有获得合格股票的相同权利和特权，只要每位员工可获得的股份数量超过微不足道的数量，并且
(III)行使期权的权利和特权不应被视为与限制性股票单位结算的权利和特权相同。
(iii)员工
就第(i)(II)款而言，“员工”一词不包括第4980E(d)(4)节所述的任何员工或任何被排除员工。

(iv)2018年之前的日历年度的特殊规则
对于2018年1月1日之前开始的任何日历年度，第(i)(II)款的适用不考虑合格股票的权利和特权是否相同。

(3)合格员工；被排除员工
就本款而言——
(A)一般规定
“合格员工”一词指任何——
(i)非被排除员工，并且
(ii)同意在本款作出的选择中满足财政部长确定为确保公司根据第24章对合格股票的预扣要求得到满足所必需的要求的个人。
(B)被排除员工
“被排除员工”一词，就任何公司而言，指任何个人——
(i)在日历年度的任何时候是1%所有者（在第416(i)(1)(B)(ii)节的含义内），或在之前10个日历年度的任何时候是此类1%所有者，
(ii)现在或以前任何时候是——
(I)该公司的首席执行官或担任该职务的个人，或
(II)该公司的首席财务官或担任该职务的个人，
(iii)与第(ii)款(I)或(II)分款所述的任何个人具有第318(a)(1)节所述的关系，或
(iv)在该纳税年度是该公司薪酬最高的4名高管之一，或在之前10个纳税年度的任何一年是该公司薪酬最高的4名高管之一，根据1934年证券交易法（如同该规则适用于该公司）的股东薪酬披露规则确定。
(4)选择
(A)作出选择的时间
对合格股票的选择应根据本款作出，不迟于员工在该股票中的权利首次可转让或不再受实质性没收风险约束（以较早发生者为准）之日起30天内，并应以与根据本款(b)作出选择类似的方式作出。

(B)限制
如果出现以下情况，则不得根据本节对任何合格股票作出选择——
(i)合格员工已根据本款(b)对该合格股票作出选择，
(ii)发行该合格股票的公司任何股票在作出选择之前的任何时候已在已建立的证券市场（根据第(1)(B)(iii)款确定）上可轻易交易，或
(iii)该公司在包含员工在该股票中的权利首次可转让或不再受实质性没收风险约束的日历年度之前的日历年度购买了其任何已发行股票，除非——
(I)所购买股票总美元金额的至少25%是递延股票，并且
(II)确定从哪些个人购买递延股票是基于合理原则进行的。
(C)与股票赎回限制相关的定义和特殊规则
(i)递延股票
就本款而言，“递延股票”一词指根据本款已生效选择的股票。

(ii)如果个人持有更长递延期的递延股票，则不考虑任何个人的递延股票
如果公司从任何个人购买的股票（紧接购买后）持有根据本款已生效选择的递延股票，且该选择的期限长于所购买股票的选择期限，则该股票不应被视为本款(B)(iii)的目的的递延股票。

(iii)购买所有已发行递延股票
如果所购买的股票包括公司所有已发行的递延股票，则本款(B)(iii)的第(I)和(II)分款的要求应被视为已满足。

(iv)报告
任何在任何日历年度开始时拥有已发行递延股票并在该日历年度内购买其任何已发行股票的公司，应在其纳税年度的报税表中包含在该日历年度内购买的已发行股票的总美元金额以及财政部长为管理本款所需的其他信息。

(5)受控集团
就本款而言，根据第414(b)节被视为单一雇主的所有人应被视为1个公司。

(6)通知要求
任何将合格股票转让给合格员工的公司，应在该股票的应税金额（若无本款）首次计入该员工总收入之时（或合理期间之前）——
(A)向该员工证明该股票是合格股票，并且
(B)通知该员工——
(i)该员工可能有资格根据本款选择递延该股票的收入，并且
(ii)如果该员工作出此项选择——
(I)在递延期结束时确认的收入金额将基于员工在该股票中的权利首次可转让或不受实质性没收风险约束时的股票价值，无论股票价值在递延期内是否下降，
(II)在递延期结束时确认的该收入金额将根据第3401(i)节按第3402(t)节确定的税率进行预扣，并且
(III)员工（根据第(3)(A)(ii)款由财政部长确定）对该预扣的责任。
(7)限制性股票单位
本节（本款除外），包括根据本款(b)作出的任何选择，不适用于限制性股票单位。
"""



######################################## 提示元素 ########################################

##### 提示元素 1：`user` 角色
# 确保您的 Messages API 调用中的消息数组始终以 `user` 角色开头。
# 上面定义的 `get_completion()` 函数会自动为您完成此操作。

##### 提示元素 2：任务上下文
# 告知 Claude 它应该扮演的角色，或者您希望它通过此提示承担的目标和总体任务。
# 最好将上下文放在提示正文的早期部分。
TASK_CONTEXT = ""

##### 提示元素 3：语气上下文
# 如果对交互很重要，请告诉 Claude 它应该使用什么语气。
# 此元素可能不是所有任务都必需的。
TONE_CONTEXT = ""

##### 提示元素 4：要处理的输入数据
# 如果提示中包含 Claude 需要处理的数据，请将其包含在此处相关的 XML 标签内。
# 您可以包含多条数据，但请务必将每条数据都包含在自己的 XML 标签集中。
# 此元素可能不是所有任务都必需的。顺序也灵活。
INPUT_DATA = ""

##### 元素 5：示例
# 为 Claude 提供至少一个它能模仿的理想响应示例。将其封装在 <example></example> XML 标签中。您可以随意提供多个示例。
# 如果您提供多个示例，请告知 Claude 这是什么示例，并将每个示例都包含在自己的 XML 标签集中。
# 示例可能是知识工作中让 Claude 表现出预期行为的最有效工具。
# 确保为 Claude 提供常见边缘情况的示例。如果您的提示使用了草稿本（scratchpad），提供草稿本应如何呈现的示例会很有效。
# 通常，示例越多越好。
EXAMPLES = ""

##### 提示元素 6：详细任务描述和规则
# 详细说明您希望 Claude 执行的具体任务，以及 Claude 可能需要遵循的任何规则。
# 您也可以在此处为 Claude 提供一个“退出”选项，如果它没有答案或不知道如何回应。
# 最好将此描述和规则展示给朋友，以确保其逻辑清晰，并且任何模糊的词语都已明确定义。
TASK_DESCRIPTION = ""

##### 提示元素 7：即时任务描述或请求 #####
# “提醒”Claude 或明确告诉 Claude 它需要立即做什么来完成提示的任务。
# 这也是您放置其他变量（如用户问题）的地方。
# 重申 Claude 的即时任务通常没有坏处。最好在长提示的末尾这样做。
# 这将比将其放在开头产生更好的结果。
# 通常，将用户查询放在提示的底部附近也是一个好习惯。
IMMEDIATE_TASK = ""

##### 提示元素 8：预认知（逐步思考）
# 对于多步骤任务，最好告诉 Claude 在给出答案之前逐步思考。
# 有时，您甚至可能需要说“在您给出答案之前...”以确保 Claude 首先执行此操作。
# 并非所有提示都必需，但如果包含，最好在长提示的末尾，紧接在最终的即时任务请求或描述之后。
PRECOGNITION = ""

##### 提示元素 9：输出格式
# 如果您希望 Claude 的回应有特定的格式，请清楚地告诉 Claude 该格式是什么。
# 此元素可能不是所有任务都必需的。
# 如果您包含它，将其放在提示的末尾比放在开头更好。
OUTPUT_FORMATTING = ""

##### 提示元素 10：预填充 Claude 的回应（如果有）
# 一个用于以预填充的词语开始 Claude 的答案的空间，以引导 Claude 的行为或回应。
# 如果您想预填充 Claude 的回应，您必须将其放入 API 调用中的 `assistant` 角色。
# 此元素可能不是所有任务都必需的。
PREFILL = ""



######################################## 组合元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n助手回合")
print(PREFILL)
print("\n------------------------------------- Claude 的回应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
from hints import exercise_9_1_solution; print(exercise_9_1_solution)

### 练习 9.2 - 代码机器人
在这个练习中，我们将为**一个能够阅读代码并在适当时候提供指导性修正的代码辅助和教学机器人**编写一个提示。请根据您在前面复杂提示示例中看到的描述和示例，填写提示元素字段的内容。填写完所有您想填写的提示元素后，运行该单元格，查看拼接后的提示以及 Claude 的回应。

我们建议您仔细阅读变量内容（在本例中是 `{CODE}`），以了解 Claude 预期处理的内容。请务必在您的提示中直接引用 `{CODE}`（使用与其他示例类似的 f-string 语法），以便实际的变量内容能够被替换进去。

In [ ]:
######################################## 输入变量 ########################################

# 输入变量 - Claude 需要阅读并协助用户纠正的代码
CODE = """
# 打印乘法逆元的函数
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## 提示元素 ########################################

##### 提示元素 1：`user` 角色
# 确保您的 Messages API 调用中的消息数组始终以 `user` 角色开头。
# 上面定义的 `get_completion()` 函数会自动为您完成此操作。

##### 提示元素 2：任务上下文
# 告知 Claude 它应该扮演的角色，或者您希望它通过此提示承担的目标和总体任务。
# 最好将上下文放在提示正文的早期部分。
TASK_CONTEXT = ""

##### 提示元素 3：语气上下文
# 如果对交互很重要，请告诉 Claude 它应该使用什么语气。
# 此元素可能不是所有任务都必需的。
TONE_CONTEXT = ""

##### 提示元素 4：详细任务描述和规则
# 详细说明您希望 Claude 执行的具体任务，以及 Claude 可能需要遵循的任何规则。
# 您也可以在此处为 Claude 提供一个“退出”选项，如果它没有答案或不知道如何回应。
# 最好将此描述和规则展示给朋友，以确保其逻辑清晰，并且任何模糊的词语都已明确定义。
TASK_DESCRIPTION = ""

##### 提示元素 5：示例
# 为 Claude 提供至少一个它能模仿的理想响应示例。将其封装在 <example></example> XML 标签中。您可以随意提供多个示例。
# 如果您提供多个示例，请告知 Claude 这是什么示例，并将每个示例都包含在自己的 XML 标签集中。
# 示例可能是知识工作中让 Claude 表现出预期行为的最有效工具。
# 确保为 Claude 提供常见边缘情况的示例。如果您的提示使用了草稿本（scratchpad），提供草稿本应如何呈现的示例会很有效。
# 通常，示例越多越好。
EXAMPLES = ""

##### 提示元素 6：要处理的输入数据
# 如果提示中包含 Claude 需要处理的数据，请将其包含在此处相关的 XML 标签内。
# 您可以包含多条数据，但请务必将每条数据都包含在自己的 XML 标签集中。
# 此元素可能不是所有任务都必需的。顺序也灵活。
INPUT_DATA = ""

##### 提示元素 7：即时任务描述或请求 #####
# “提醒”Claude 或明确告诉 Claude 它需要立即做什么来完成提示的任务。
# 这也是您放置其他变量（如用户问题）的地方。
# 重申 Claude 的即时任务通常没有坏处。最好在长提示的末尾这样做。
# 这将比将其放在开头产生更好的结果。
# 通常，将用户查询放在提示的底部附近也是一个好习惯。
IMMEDIATE_TASK = ""

##### 提示元素 8：预认知（逐步思考）
# 对于多步骤任务，最好告诉 Claude 在给出答案之前逐步思考。
# 有时，您甚至可能需要说“在您给出答案之前...”以确保 Claude 首先执行此操作。
# 并非所有提示都必需，但如果包含，最好在长提示的末尾，紧接在最终的即时任务请求或描述之后。
PRECOGNITION = ""

##### 提示元素 9：输出格式
# 如果您希望 Claude 的回应有特定的格式，请清楚地告诉 Claude 该格式是什么。
# 此元素可能不是所有任务都必需的。
# 如果您包含它，将其放在提示的末尾比放在开头更好。
OUTPUT_FORMATTING = ""

##### 提示元素 10：预填充 Claude 的回应（如果有）
# 一个用于以预填充的词语开始 Claude 的答案的空间，以引导 Claude 的行为或回应。
# 如果您想预填充 Claude 的回应，您必须将其放入 API 调用中的 `assistant` 角色。
# 此元素可能不是所有任务都必需的。
PREFILL = ""



######################################## 组合元素 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 打印完整提示
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n助手回合")
print(PREFILL)
print("\n------------------------------------- Claude 的回应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
from hints import exercise_9_2_solution; print(exercise_9_2_solution)

---

## 恭喜与下一步！

如果您完成了所有练习，**您现在已跻身顶尖的 0.1% 大型语言模型“耳语者”之列**。精英中的精英！

您所学的技巧，从逐步思考到分配角色，再到使用示例以及全面的清晰写作，都可以**以无数种方式进行融合、重组和调整**。

提示工程是一个非常新的学科，所以请保持开放的心态。您可能就是发现下一个伟大提示技巧的人。

如果您想查看**更多优秀提示示例**以获取灵感：
- 从我们的[烹饪书（cookbook）](https://anthropic.com/cookbook)中学习生产级提示的示例
- 阅读我们的[提示指南](https://docs.anthropic.com/claude/docs/prompt-engineering)
- 查阅我们的[提示库](https://anthropic.com/prompts)以获取灵感
- 尝试我们的实验性[元提示（metaprompt）](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental)，让 Claude 为您编写提示模板！
- 在我们的[Discord 服务器](https://anthropic.com/discord)中提问
- 了解 [Anthropic API 参数](https://docs.anthropic.com/claude/reference/complete_post)，如 `temperature` 和 `max_tokens`
- 如果您有学术兴趣，可以阅读一些关于提示工程的[论文](https://www.promptingguide.ai/papers)
- 练习构建提示，让 Claude 完成您感兴趣的事情

如果您想了解本教程范围之外的一些真正高级的提示技术，请点击进入附录！但在此之前，请运行下面的单元格。

In [ ]:
# 提示
PROMPT = "请为一位刚刚完成提示工程课程的优秀学生写一首颂歌，形式为十四行诗。"

# 打印 Claude 的回应
print(get_completion(PROMPT))

---

# Chapter 9: Complex Prompts from Scratch

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

Congratulations on making it to the last chapter! Now time to put everything together and learn how to **create unique and complex prompts**. 

Below, you will be using a **guided structure that we recommend for complex prompts**. In latter parts of this chapter, we will show you some industry-specific prompts and explain how those prompts are similarly structured.

**Note:** **Not all prompts need every element of the following complex structure**. We encourage you to play around with and include or disinclude elements and see how it affects Claude's response. It is usually **best to use many prompt elements to get your prompt working first, then refine and slim down your prompt afterward**.

### Example - Career Coach Chatbot

The following structure combines multiple prompt engineering elements and is a good starting point for complex prompts. **The ordering matters for some elements**, not for others. We will note when best practices indicate ordering matters, but in general, **if you stick to this ordering, it will be a good start to a stellar prompt**.

For the following example, we will be building a prompt for a controlled roleplay wherein Claude takes on a situational role with a specific task. Our goal is to prompt Claude to act as a friendly career coach.

Read then run the cell below to compile the various prompt elements into one whole prompt.

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Second input variable - the user's question
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Think about your answer first before you respond."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "[Joe] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

Now let's run the prompt! Run the cell below to see Claude's output.

In [ ]:
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

### Example - Legal Services

**Prompts within the legal profession can be quite complex** due to the need to:
- Parse long documents
- Deal with complex topics
- Format output in very specific ways
- Follow multi-step analytical processes

Let's see how we can use the complex prompt template to structure a prompt for a specific legal use-case. Below, we've detailed out an example prompt for a legal use-case wherein we ask Claude to answer questions about a legal issue using information from a legal document.

We've **changed around the ordering of a few elements** to showcase that prompt structure can be flexible!

**Prompt engineering is about scientific trial and error**. We encourage you to mix and match, move things around (the elements where ordering doesn't matter), and see what works best for you and your needs. 

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = f"""Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

Now let's run the prompt! Run the cell below to see Claude's output.

In [ ]:
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 9.1 - Financial Services Chatbot](#exercise-91---financial-services-chatbot)
- [Exercise 9.2 - Codebot](#exercise-92---codebot)

### Exercise 9.1 - Financial Services Chatbot
Prompts within the financial profession can also be quite complex due to reasons similar to legal prompts. Here's an exercise for a financial use-case, wherein Claude is used to **analyze tax information and answer questions**. Just like with the legal services example, we've changed around the ordering of a few elements, as our solution prompt makes more sense with a different flow (however, other structures would also work).

We suggest you read through the variable content (in this case, `{QUESTION}` and `{TAX_CODE}`) to understand what content Claude is expected to work with. Be sure to reference `{QUESTION}` and `{TAX_CODE}` directly in your prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

Fill in the prompt element fields with content that match the description and the examples you've seen in the preceding examples of complex prompts. Once you have filled out all the prompt elements that you want to fill out, run the cell to see the concatenated prompt as well as Claude's response.

Remember that prompt engineering is rarely purely formulaic, especially for large and complex prompts! It's important to develop test cases and **try a variety of prompts and prompt structures to see what works best for each situation**. Note that if you *do* change the ordering of the prompt elements, you should also remember to change the ordering of the concatenaton in the `COMBINE ELEMENTS` section.

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the user's question
QUESTION = "How long do I have to make an 83b election?"

# Second input variable - the tax code document that Claude will be using to answer the user's question
TAX_CODE = """
(a)General rule
If, in connection with the performance of services, property is transferred to any person other than the person for whom such services are performed, the excess of—
(1)the fair market value of such property (determined without regard to any restriction other than a restriction which by its terms will never lapse) at the first time the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, over
(2)the amount (if any) paid for such property,
shall be included in the gross income of the person who performed such services in the first taxable year in which the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever is applicable. The preceding sentence shall not apply if such person sells or otherwise disposes of such property in an arm’s length transaction before his rights in such property become transferable or not subject to a substantial risk of forfeiture.
(b)Election to include in gross income in year of transfer
(1)In general
Any person who performs services in connection with which property is transferred to any person may elect to include in his gross income for the taxable year in which such property is transferred, the excess of—
(A)the fair market value of such property at the time of transfer (determined without regard to any restriction other than a restriction which by its terms will never lapse), over
(B)the amount (if any) paid for such property.
If such election is made, subsection (a) shall not apply with respect to the transfer of such property, and if such property is subsequently forfeited, no deduction shall be allowed in respect of such forfeiture.
(2)Election
An election under paragraph (1) with respect to any transfer of property shall be made in such manner as the Secretary prescribes and shall be made not later than 30 days after the date of such transfer. Such election may not be revoked except with the consent of the Secretary.

(c)Special rules
For purposes of this section—
(1)Substantial risk of forfeiture
The rights of a person in property are subject to a substantial risk of forfeiture if such person’s rights to full enjoyment of such property are conditioned upon the future performance of substantial services by any individual.

(2)Transferability of property
The rights of a person in property are transferable only if the rights in such property of any transferee are not subject to a substantial risk of forfeiture.

(3)Sales which may give rise to suit under section 16(b) of the Securities Exchange Act of 1934
So long as the sale of property at a profit could subject a person to suit under section 16(b) of the Securities Exchange Act of 1934, such person’s rights in such property are—
(A)subject to a substantial risk of forfeiture, and
(B)not transferable.
(4)For purposes of determining an individual’s basis in property transferred in connection with the performance of services, rules similar to the rules of section 72(w) shall apply.
(d)Certain restrictions which will never lapse
(1)Valuation
In the case of property subject to a restriction which by its terms will never lapse, and which allows the transferee to sell such property only at a price determined under a formula, the price so determined shall be deemed to be the fair market value of the property unless established to the contrary by the Secretary, and the burden of proof shall be on the Secretary with respect to such value.

(2)Cancellation
If, in the case of property subject to a restriction which by its terms will never lapse, the restriction is canceled, then, unless the taxpayer establishes—
(A)that such cancellation was not compensatory, and
(B)that the person, if any, who would be allowed a deduction if the cancellation were treated as compensatory, will treat the transaction as not compensatory, as evidenced in such manner as the Secretary shall prescribe by regulations,
the excess of the fair market value of the property (computed without regard to the restrictions) at the time of cancellation over the sum of—
(C)the fair market value of such property (computed by taking the restriction into account) immediately before the cancellation, and
(D)the amount, if any, paid for the cancellation,
shall be treated as compensation for the taxable year in which such cancellation occurs.
(e)Applicability of section
This section shall not apply to—
(1)a transaction to which section 421 applies,
(2)a transfer to or from a trust described in section 401(a) or a transfer under an annuity plan which meets the requirements of section 404(a)(2),
(3)the transfer of an option without a readily ascertainable fair market value,
(4)the transfer of property pursuant to the exercise of an option with a readily ascertainable fair market value at the date of grant, or
(5)group-term life insurance to which section 79 applies.
(f)Holding period
In determining the period for which the taxpayer has held property to which subsection (a) applies, there shall be included only the period beginning at the first time his rights in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier.

(g)Certain exchanges
If property to which subsection (a) applies is exchanged for property subject to restrictions and conditions substantially similar to those to which the property given in such exchange was subject, and if section 354, 355, 356, or 1036 (or so much of section 1031 as relates to section 1036) applied to such exchange, or if such exchange was pursuant to the exercise of a conversion privilege—
(1)such exchange shall be disregarded for purposes of subsection (a), and
(2)the property received shall be treated as property to which subsection (a) applies.
(h)Deduction by employer
In the case of a transfer of property to which this section applies or a cancellation of a restriction described in subsection (d), there shall be allowed as a deduction under section 162, to the person for whom were performed the services in connection with which such property was transferred, an amount equal to the amount included under subsection (a), (b), or (d)(2) in the gross income of the person who performed such services. Such deduction shall be allowed for the taxable year of such person in which or with which ends the taxable year in which such amount is included in the gross income of the person who performed such services.

(i)Qualified equity grants
(1)In general
For purposes of this subtitle—
(A)Timing of inclusion
If qualified stock is transferred to a qualified employee who makes an election with respect to such stock under this subsection, subsection (a) shall be applied by including the amount determined under such subsection with respect to such stock in income of the employee in the taxable year determined under subparagraph (B) in lieu of the taxable year described in subsection (a).

(B)Taxable year determined
The taxable year determined under this subparagraph is the taxable year of the employee which includes the earliest of—
(i)the first date such qualified stock becomes transferable (including, solely for purposes of this clause, becoming transferable to the employer),
(ii)the date the employee first becomes an excluded employee,
(iii)the first date on which any stock of the corporation which issued the qualified stock becomes readily tradable on an established securities market (as determined by the Secretary, but not including any market unless such market is recognized as an established securities market by the Secretary for purposes of a provision of this title other than this subsection),
(iv)the date that is 5 years after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, or
(v)the date on which the employee revokes (at such time and in such manner as the Secretary provides) the election under this subsection with respect to such stock.
(2)Qualified stock
(A)In general
For purposes of this subsection, the term “qualified stock” means, with respect to any qualified employee, any stock in a corporation which is the employer of such employee, if—
(i)such stock is received—
(I)in connection with the exercise of an option, or
(II)in settlement of a restricted stock unit, and
(ii)such option or restricted stock unit was granted by the corporation—
(I)in connection with the performance of services as an employee, and
(II)during a calendar year in which such corporation was an eligible corporation.
(B)Limitation
The term “qualified stock” shall not include any stock if the employee may sell such stock to, or otherwise receive cash in lieu of stock from, the corporation at the time that the rights of the employee in such stock first become transferable or not subject to a substantial risk of forfeiture.

(C)Eligible corporation
For purposes of subparagraph (A)(ii)(II)—
(i)In general
The term “eligible corporation” means, with respect to any calendar year, any corporation if—
(I)no stock of such corporation (or any predecessor of such corporation) is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) during any preceding calendar year, and
(II)such corporation has a written plan under which, in such calendar year, not less than 80 percent of all employees who provide services to such corporation in the United States (or any possession of the United States) are granted stock options, or are granted restricted stock units, with the same rights and privileges to receive qualified stock.
(ii)Same rights and privileges
For purposes of clause (i)(II)—
(I)except as provided in subclauses (II) and (III), the determination of rights and privileges with respect to stock shall be made in a similar manner as under section 423(b)(5),
(II)employees shall not fail to be treated as having the same rights and privileges to receive qualified stock solely because the number of shares available to all employees is not equal in amount, so long as the number of shares available to each employee is more than a de minimis amount, and
(III)rights and privileges with respect to the exercise of an option shall not be treated as the same as rights and privileges with respect to the settlement of a restricted stock unit.
(iii)Employee
For purposes of clause (i)(II), the term “employee” shall not include any employee described in section 4980E(d)(4) or any excluded employee.

(iv)Special rule for calendar years before 2018
In the case of any calendar year beginning before January 1, 2018, clause (i)(II) shall be applied without regard to whether the rights and privileges with respect to the qualified stock are the same.

(3)Qualified employee; excluded employee
For purposes of this subsection—
(A)In general
The term “qualified employee” means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.
(B)Excluded employee
The term “excluded employee” means, with respect to any corporation, any individual—
(i)who is a 1-percent owner (within the meaning of section 416(i)(1)(B)(ii)) at any time during the calendar year or who was such a 1 percent owner at any time during the 10 preceding calendar years,
(ii)who is or has been at any prior time—
(I)the chief executive officer of such corporation or an individual acting in such a capacity, or
(II)the chief financial officer of such corporation or an individual acting in such a capacity,
(iii)who bears a relationship described in section 318(a)(1) to any individual described in subclause (I) or (II) of clause (ii), or
(iv)who is one of the 4 highest compensated officers of such corporation for the taxable year, or was one of the 4 highest compensated officers of such corporation for any of the 10 preceding taxable years, determined with respect to each such taxable year on the basis of the shareholder disclosure rules for compensation under the Securities Exchange Act of 1934 (as if such rules applied to such corporation).
(4)Election
(A)Time for making election
An election with respect to qualified stock shall be made under this subsection no later than 30 days after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, and shall be made in a manner similar to the manner in which an election is made under subsection (b).

(B)Limitations
No election may be made under this section with respect to any qualified stock if—
(i)the qualified employee has made an election under subsection (b) with respect to such qualified stock,
(ii)any stock of the corporation which issued the qualified stock is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) at any time before the election is made, or
(iii)such corporation purchased any of its outstanding stock in the calendar year preceding the calendar year which includes the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, unless—
(I)not less than 25 percent of the total dollar amount of the stock so purchased is deferral stock, and
(II)the determination of which individuals from whom deferral stock is purchased is made on a reasonable basis.
(C)Definitions and special rules related to limitation on stock redemptions
(i)Deferral stock
For purposes of this paragraph, the term “deferral stock” means stock with respect to which an election is in effect under this subsection.

(ii)Deferral stock with respect to any individual not taken into account if individual holds deferral stock with longer deferral period
Stock purchased by a corporation from any individual shall not be treated as deferral stock for purposes of subparagraph (B)(iii) if such individual (immediately after such purchase) holds any deferral stock with respect to which an election has been in effect under this subsection for a longer period than the election with respect to the stock so purchased.

(iii)Purchase of all outstanding deferral stock
The requirements of subclauses (I) and (II) of subparagraph (B)(iii) shall be treated as met if the stock so purchased includes all of the corporation’s outstanding deferral stock.

(iv)Reporting
Any corporation which has outstanding deferral stock as of the beginning of any calendar year and which purchases any of its outstanding stock during such calendar year shall include on its return of tax for the taxable year in which, or with which, such calendar year ends the total dollar amount of its outstanding stock so purchased during such calendar year and such other information as the Secretary requires for purposes of administering this paragraph.

(5)Controlled groups
For purposes of this subsection, all persons treated as a single employer under section 414(b) shall be treated as 1 corporation.

(6)Notice requirement
Any corporation which transfers qualified stock to a qualified employee shall, at the time that (or a reasonable period before) an amount attributable to such stock would (but for this subsection) first be includible in the gross income of such employee—
(A)certify to such employee that such stock is qualified stock, and
(B)notify such employee—
(i)that the employee may be eligible to elect to defer income on such stock under this subsection, and
(ii)that, if the employee makes such an election—
(I)the amount of income recognized at the end of the deferral period will be based on the value of the stock at the time at which the rights of the employee in such stock first become transferable or not subject to substantial risk of forfeiture, notwithstanding whether the value of the stock has declined during the deferral period,
(II)the amount of such income recognized at the end of the deferral period will be subject to withholding under section 3401(i) at the rate determined under section 3402(t), and
(III)the responsibilities of the employee (as determined by the Secretary under paragraph (3)(A)(ii)) with respect to such withholding.
(7)Restricted stock units
This section (other than this subsection), including any election under subsection (b), shall not apply to restricted stock units.
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ If you want to see a possible solution, run the cell below!

In [ ]:
from hints import exercise_9_1_solution; print(exercise_9_1_solution)

### Exercise 9.2 - Codebot
In this exercise, we will write up a prompt for a **coding assistance and teaching bot that reads code and offers guiding corrections when appropriate**. Fill in the prompt element fields with content that match the description and the examples you've seen in the preceding examples of complex prompts. Once you have filled out all the prompt elements that you want to fill out, run the cell to see the concatenated prompt as well as Claude's response.

We suggest you read through the variable content (in this case, `{CODE}`) to understand what content Claude is expected to work with. Be sure to reference `{CODE}` directly in your prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

In [ ]:
######################################## INPUT VARIABLES ########################################

# Input variable - the code that Claude needs to read and assist the user with correcting
CODE = """
# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ If you want to see a possible solution, run the cell below!

In [ ]:
from hints import exercise_9_2_solution; print(exercise_9_2_solution)

---

## Congratulations & Next Steps!

If you made it through all the exercises, **you are now in the top 0.1% of LLM whisperers**. One of the elite!

The techniques you've learned, from thinking step by step to assigning roles to using examples to general all-around clear writing, can be **merged, remixed, and adapted in countless ways**.

Prompt engineering is a very new discipline, so keep an open mind. You could be the one to discover the next great prompting trick.

If you want to see **more examples of good prompts** for inspiration:					
- Learn from examples of production-ready prompts from our [cookbook](https://anthropic.com/cookbook)
- Read through our [prompting guide](https://docs.anthropic.com/claude/docs/prompt-engineering)
- Check out our [prompt library](https://anthropic.com/prompts) for inspiration
- Try our experimental [metaprompt](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental) to get Claude to write prompt templates for you!
- Ask questions in our [discord server](https://anthropic.com/discord)
- Learn about the [Anthropic API parameters](https://docs.anthropic.com/claude/reference/complete_post) like temperature and `max_tokens`
- If you're feeling academic, read some [papers](https://www.promptingguide.ai/papers) on prompt engineering
- Practice building prompts to get Claude to do something you're interested in

If you want to learn about some truly advanced prompting techniques beyond the scope of this tutorial, click through to the appendix! But first, run the cell below.

In [ ]:
# Prompt
PROMPT = "Write an ode to a fabulous student who has just completed a course on prompt engineering, in the form of a sonnet."

# Print Claude's response
print(get_completion(PROMPT))

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Second input variable - the user's question
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Think about your answer first before you respond."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "[Joe] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))